In [1]:
# Blackjack

import simplegui
import random

# load card sprite - 936x384 - source: jfitz.com
CARD_SIZE = (72, 96)
CARD_CENTER = (36, 48)
card_images = simplegui.load_image("http://storage.googleapis.com/codeskulptor-assets/cards_jfitz.png")

CARD_BACK_SIZE = (72, 96)
CARD_BACK_CENTER = (36, 48)
card_back = simplegui.load_image("http://storage.googleapis.com/codeskulptor-assets/card_jfitz_back.png")    

# initialize some useful global variables
in_play = False
outcome = ""
score = 0

# define globals for cards
SUITS = ('C', 'S', 'H', 'D')
RANKS = ('A', '2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K')
VALUES = {'A':1, '2':2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 'T':10, 'J':10, 'Q':10, 'K':10}

# define card class
class Card:
    def __init__(self, suit, rank):
        if (suit in SUITS) and (rank in RANKS):
            self.suit = suit
            self.rank = rank
        else:
            self.suit = None
            self.rank = None
            print "Invalid card: ", suit, rank

    def __str__(self):
        return self.suit + self.rank

    def get_suit(self):
        return self.suit

    def get_rank(self):
        return self.rank

    def draw(self, canvas, pos):
        card_loc = (CARD_CENTER[0] + CARD_SIZE[0] * RANKS.index(self.rank), 
                    CARD_CENTER[1] + CARD_SIZE[1] * SUITS.index(self.suit))
        canvas.draw_image(card_images, card_loc, CARD_SIZE, [pos[0] + CARD_CENTER[0], pos[1] + CARD_CENTER[1]], CARD_SIZE)
        
# define hand class
class Hand:
    def __init__(self):
        self.hand = [] # create Hand object

    def __str__(self):
        ans = ""
        for i in range(len(self.hand)):
            ans += str(self.hand[i]) + " " 
        return 'Hand contains ' + ans	# return a string representation of a hand

    def add_card(self, card):
        self.hand.append(card) # add a card object to a hand

    def get_value(self):
        hand_value = 0
        card_hand = []
        n = 0
        # calculate the hand value
        for card in self.hand:
            for key, value in VALUES.items():
                if card.get_rank() == key:
                    card_value = value
                    hand_value += card_value
                    card_hand.append(key)

        if "A" not in card_hand:
            hand_value = hand_value
        else:
            if hand_value + 10 <= 21:
                hand_value = hand_value + 10
            else:
                hand_value = hand_value
        return hand_value

    def draw(self, canvas, pos):
        for card in self.hand:
            card.draw(canvas, pos)
            pos[0] += 60
        
# define deck class 
# create a Deck object
class Deck:
    def __init__(self):
        self.deck = []
        for suit in SUITS:
            for rank in RANKS:
                self.deck.append(Card(suit, rank))        

    def shuffle(self):
        # shuffle the deck 
        random.shuffle(self.deck)
        
    def deal_card(self):
        # deal a card object from the deck
        d_card = random.choice(self.deck)
        ind = self.deck.index(d_card)
        return self.deck.pop(ind)        
    
    def __str__(self):
        # return a string representing the deck
        ans = ""
        for i in range(len(self.deck)):
            ans += str(self.deck[i]) + " " 
        return 'Deck contains ' + ans

# define event handlers for buttons
def deal():
    global outcome, in_play, Dealer_Hand, Player_Hand, deck, score
    # initialize deck
    if in_play == True:
        score -= 1
        outcome = ""
        deck = Deck()
        deck.shuffle()
        # create Player and Dealer Hands
        Player_Hand = Hand()
        Dealer_Hand = Hand()
        # transfer two cards to both the player and the dealer from deck
        Player_Hand.add_card(deck.deal_card())
        Player_Hand.add_card(deck.deal_card())
        Dealer_Hand.add_card(deck.deal_card())
        Dealer_Hand.add_card(deck.deal_card())
        print "Player " + str(Player_Hand) + "Dealer " + str(Dealer_Hand)
        in_play = True
    else:
        outcome = ""
        deck = Deck()
        deck.shuffle()
        # create Player and Dealer Hands
        Player_Hand = Hand()
        Dealer_Hand = Hand()
        # transfer two cards to both the player and the dealer from deck
        Player_Hand.add_card(deck.deal_card())
        Player_Hand.add_card(deck.deal_card())
        Dealer_Hand.add_card(deck.deal_card())
        Dealer_Hand.add_card(deck.deal_card())
        print "Player " + str(Player_Hand) + "Dealer " + str(Dealer_Hand)
        in_play = True

def hit():
    global outcome, in_play, deck, Player_Hand, Dealer_Hand, score
    if Player_Hand.get_value() <= 21:
        Player_Hand.add_card(deck.deal_card())
    else:
        outcome = "You have busted! New deal?"
        in_play = False
        score -= 1
                
def stand():
    global score, outcome, in_play
    if in_play == False:
        outcome = "You have busted! New deal?"
        score -= 1
    else:
        while Dealer_Hand.get_value() <= 17:
            Dealer_Hand.add_card(deck.deal_card())
        if Dealer_Hand.get_value() > 21:
            outcome = "Dealer has busted! New deal?"
            in_play = False
            score += 1
        else:
            # Dealer wins!
            if Dealer_Hand.get_value() >= Player_Hand.get_value():
                outcome = "You have busted! New deal?"
                in_play = False
                score -= 1
            # Player wins!
            else:
                outome = "Dealer has busted! New deal?"
                in_play = False
                score += 1

# draw handler    
def draw(canvas):
    canvas.draw_text("BlackJack", [100, 100], 40, "Black")
    canvas.draw_text("Dealer", [120, 180], 30, "Black")
    canvas.draw_text("Player", [120, 380], 30, "Black")
    canvas.draw_text("Score "+str(score), [350, 100], 25, "Black")
    Dealer_Hand.draw(canvas, [100, 200])
    Player_Hand.draw(canvas, [100, 400])
    canvas.draw_text(outcome, [250, 180], 25, "Black")
    if in_play == True:
        canvas.draw_image(card_back, CARD_BACK_CENTER, CARD_BACK_SIZE, [136, 248], CARD_BACK_SIZE)
    else:
        pass
        
# initialization frame
frame = simplegui.create_frame("Blackjack", 600, 600)
frame.set_canvas_background("Green")

# create buttons and canvas callback
frame.add_button("Deal", deal, 200)
frame.add_button("Hit",  hit, 200)
frame.add_button("Stand", stand, 200)
frame.set_draw_handler(draw)

# get things rolling
deal()	
frame.start()

# http://www.codeskulptor.org/#user46_kl00UZYR9E_2.py

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Invalid card: ", suit, rank)? (<ipython-input-1-a4fcdc59f981>, line 34)